<a href="https://colab.research.google.com/github/rudolmat/SAKI-Exercise1/blob/master/3_Screenshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/My Drive/SAKI/Exercise_2

/content/gdrive/My Drive/SAKI/Exercise_2


In [0]:
import os
import json
dataset_path = "DataSet.json"
#print("Path exists? {}".format(os.path.exists(dataset_path)))

In [0]:
## use the "open" function to get a filehandle. 
with open(dataset_path,encoding="utf8") as f:
    doc = f.readlines()
    
all_resumes = []
for line in doc:
    all_resumes.append(json.loads(line))

In [15]:
## data conversion method
def convert_data(data):
    """
    Creates NER training data in Spacy format from JSON dataset
    Outputs the Spacy training data which can be used for Spacy training.
    """
    text = data['content']
    entities = []
    if data['annotation'] is not None:
        for annotation in data['annotation']:
            # only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]
            for label in labels:
                # dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1, label))
    # become list of Content string, "entities" start end "label", start end "label"...
    # maybe clean empty rows?
    return (text, {"entities": entities})
   
## TODO using a loop or list comprehension, convert each resume 
#in all_resumes using the convert function above, storing the result
converted_resumes = []
converted_resumes = [convert_data(line) for line in all_resumes]

# filter out resumes where resume entities list is None

converted_resumes = [line for line in converted_resumes if len(line[1]["entities"]) > 0]
print(len(converted_resumes))

690


In [16]:
## collect names of all entities in complete resume dataset
all_labels = list()
for res in converted_resumes:
    ## entity list of res
    for ents in res[1]['entities']:
        all_labels.append(ents[2])  
        
# Make labels a set of unique values
unique_labels = set(all_labels)
## Print unique entity labels
print("Entity labels: ",unique_labels)
print(len(unique_labels))

Entity labels:  {'Location', 'College', 'projects', 'Certifications', 'UNKNOWN', 'Years of Experience', 'Rewards and Achievements', 'Graduation Year', 'Name', 'University', 'links', 'Designation', 'state', 'College Name', 'Relocate to', 'abc', 'des', 'Links', 'Companies worked at', 'Email Address', 'training', 'Can Relocate to', 'Skills', 'Address', 'Degree'}
25


In [0]:
#Create File as reference
converted_resumes_path = r"C:\Users\matth\Documents\Uni\Master\Kurse\SAKI\Exercise_2_fin.json"
with open(converted_resumes_path,"w", encoding = "UTF-8") as file_write:
    json.dump(converted_resumes,file_write)

Preprocessing of Data

In [0]:
import spacy
from spacy_train_resume_ner import train_spacy_ner
path = r"Exercise_2_fin.json"
with open(path) as json_file:
    resumes = json.load(json_file)

In [19]:
## Entities for training:
chosen_entity_labels = ["Degree", "Designation","Companies worked at"]

## check if the chosen entities are sufficient
for chosen in chosen_entity_labels:
    found_docs_with_entity = 0
    entity_count = 0
    for resume in converted_resumes:
        entity_list = resume[1]["entities"]
        _,_,labels = zip(*entity_list)
        if chosen in labels:
            found_docs_with_entity+=1
            entity_count+=len([l for l in labels if l == chosen])
    print("Docs with {}: {}".format(chosen,found_docs_with_entity))
    print("Total count of {}: {}".format(chosen,entity_count))

#gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates
#store gathered resumes for training
training_data = gather_candidates(resumes, chosen_entity_labels)
print("Gathered {} training examples".format(len(training_data)))
#print(training_data)

Docs with Degree: 606
Total count of Degree: 1012
Docs with Designation: 650
Total count of Designation: 2842
Docs with Companies worked at: 627
Total count of Companies worked at: 2830
Gathered 547 training examples


In [20]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

X = [filter_ents(line[1]["entities"],chosen_entity_labels) for line in training_data]
for i in range(0,len(training_data)):
    training_data[i][1]["entities"]=X[i]
    
#remove additional bad data    
def remove_bad_data(training_data):
    model, baddocs = train_spacy_ner(training_data, debug=True, n_iter=1)
    filtered = [data for data in training_data if data[0] not in baddocs]
    print("Unfiltered training data size: ",len(training_data))
    print("Filtered training data size: ", len(filtered))
    print("Bad data size: ", len(baddocs))
    return filtered

## call remove method 
X = remove_bad_data(training_data)

print("amount of resumes used:", len(X))

Created blank 'en' model
Exception thrown when processing doc:
('Nida Khan\nTech Support Executive - Teleperformance for Microsoft\n\nJaipur, Rajasthan - Email me on Indeed: indeed.com/r/Nida-Khan/6c9160696f57efd8\n\n• To be an integral part of the organization and enhance my knowledge to utilize it in a productive\nmanner for the growth of the company and the global.\n\nINDUSTRIAL TRAINING\n\n• BHEL, (HEEP) HARIDWAR\nOn CNC System&amp; PLC Programming.\n\nWORK EXPERIENCE\n\nTech Support Executive\n\nTeleperformance for Microsoft -\n\nSeptember 2017 to Present\n\nprocess.\n• 21 months of experience in ADFC as Phone Banker.\n\nEDUCATION\n\nBachelor of Technology in Electronics & communication Engg\n\nGNIT institute of Technology -  Lucknow, Uttar Pradesh\n\n2008 to 2012\n\nClass XII\n\nU.P. Board -  Bareilly, Uttar Pradesh\n\n2007\n\nClass X\n\nU.P. Board -  Bareilly, Uttar Pradesh\n\n2005\n\nSKILLS\n\nMicrosoft office, excel, cisco, c language, cbs. (4 years)\n\nhttps://www.indeed.com/

Train Test Split

In [0]:
import math

def train_test_split(X,train_percent):
    train_size = math.ceil(train_percent*len(X))
    train = X[:train_size]
    test = X[train_size:len(X)]
    assert len(train) + len(test) == len(X)
    return train,test

#  storing results in "train" and "test" variables.
train,test = train_test_split(X,0.8)


Training with spacy:

In [24]:
custom_nlp,_= train_spacy_ner(train,n_iter=25)

Created blank 'en' model
Losses {'ner': 31968.13767113188}
Losses {'ner': 30228.88540965278}
Losses {'ner': 32919.31922331465}
Losses {'ner': 29931.20157718868}
Losses {'ner': 22511.304856944986}
Losses {'ner': 16026.407853444325}
Losses {'ner': 15265.29558838547}
Losses {'ner': 9321.272257674671}
Losses {'ner': 7936.275811117183}
Losses {'ner': 7390.836710034936}
Losses {'ner': 6677.588832632146}
Losses {'ner': 6137.447280326258}
Losses {'ner': 5651.897764431078}
Losses {'ner': 5461.901170262701}
Losses {'ner': 5242.662501492144}
Losses {'ner': 5569.673595578974}
Losses {'ner': 5138.93379673189}
Losses {'ner': 4824.357937028413}
Losses {'ner': 5164.173623887769}
Losses {'ner': 5191.560264433296}
Losses {'ner': 4566.599843960386}
Losses {'ner': 4079.5635391785554}
Losses {'ner': 4858.4511767389795}
Losses {'ner': 4148.24659953339}
Losses {'ner': 3950.447255169211}


Formatting data into iob format

In [0]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc,
                                                   [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])
    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    #bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    #transform bilou format into iob format
    bilou_df["True"].replace(to_replace ="L-",value= "I-",regex=True, inplace=True)
    bilou_df["Predicted"].replace(to_replace ="L-",value= "I-",regex=True, inplace=True)
    bilou_df["True"].replace(to_replace ="U-",value= "I-",regex=True, inplace=True)
    bilou_df["Predicted"].replace(to_replace ="U-",value= "I-",regex=True, inplace=True)
    bilou_df["Predicted"].replace(to_replace =" ",value= "_",regex=True, inplace=True)
    bilou_df["True"].replace(to_replace =" ",value= "_",regex=True, inplace=True)
    return bilou_df 
    
## store a resume from test set
bilou_df = make_bilou_df(custom_nlp,resume)

Evaluation of SpaCy model

In [27]:
import numpy as np
doc_accuracy = []
chosen_entity_labels = ["Degree", "Designation","Companies_worked_at"]
#calculate accuracy
for res in test:
    ## for each 'res' 
    bilou_df = make_bilou_df(custom_nlp,res)
    same_df = bilou_df[bilou_df["Predicted"] == bilou_df["True"]]
    accuracy = len(same_df)/len(bilou_df)
    doc_accuracy.append(accuracy)
# average accuracy
total_acc = np.average(doc_accuracy)
print("Accuracy: ",total_acc)

#calculating scores for each entity
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for tres in test:        
        tres_df = make_bilou_df(custom_nlp,tres)
               
        ## calculate true false positives and false negatives for each resume
        tp = tres_df[(tres_df["Predicted"]== tres_df["True"]) & (tres_df["Predicted"].str.contains(label))]
        fp = tres_df[(tres_df["Predicted"] != tres_df["True"]) & (tres_df["Predicted"].str.contains(label))]
        fn = tres_df[(tres_df["Predicted"] != tres_df["True"]) & (tres_df["True"].str.contains(label))]
        ## aggregate result for each resume to totals
        true_positives += tp.shape[0]
        false_positives += fp.shape[0]
        false_negatives += fn.shape[0]

    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    precision = true_positives /(true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2*((precision * recall) / (precision + recall))
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("F1: ",f1)
    row = [precision,recall,f1]
    data.append(row)

## pandas dataframe with metrics data.
metric_df = pd.DataFrame(data, index = chosen_entity_labels,columns =["precision","recall","f1"] )
display(metric_df)

## Averaged metrics
avg_metr = pd.DataFrame.mean(metric_df)
display(avg_metr)

Accuracy:  0.9657573570458512
For label 'Degree' tp: 415 fp: 330 fn: 59
Precision:  0.5570469798657718
Recall:  0.8755274261603375
F1:  0.6808859721082854
For label 'Designation' tp: 834 fp: 418 fn: 269
Precision:  0.6661341853035144
Recall:  0.7561196736174071
F1:  0.7082802547770701
For label 'Companies_worked_at' tp: 925 fp: 400 fn: 345
Precision:  0.6981132075471698
Recall:  0.7283464566929134
F1:  0.7129094412331405


,precision,recall,f1
Degree,0.557047,0.875527,0.680886
Designation,0.666134,0.756120,0.708280
Companies_worked_at,0.698113,0.728346,0.712909


precision    0.640431
recall       0.786665
f1           0.700692
dtype: float64

Create files and remove inconsistencies

In [0]:
## TODO persist IOB data as text
import re
print("Make bilou dfs")
training_data_as_bilou = [make_bilou_df(custom_nlp,res) for res in train]
test_data_as_bilou = [make_bilou_df(custom_nlp,res) for res in test]


print("Done!")
training_df = pd.DataFrame(columns = ["text","ner","doc","ner_spacy"])
test_df = pd.DataFrame(columns = ["text","ner","doc","ner_spacy"])
for idx,df in enumerate(training_data_as_bilou):
    df2 = pd.DataFrame()
    df2["text"] = df["Tokens"]
    df2["ner"] = df["True"]
    df2["ner_spacy"] = df["Predicted"]
    df2["doc"] = idx
    training_df = training_df.append(df2)

training_df=training_df[training_df["ner"]!="-"]

#Sentence seperator
training_df.loc[(training_df["text"]==""),'ner_spacy']=""
training_df.loc[(training_df["text"]==""),'ner']=""
training_df.loc[(training_df["text"]==""),'doc']=""


for idx,df in enumerate(test_data_as_bilou):
     df2 = pd.DataFrame()
     df2["text"] = df["Tokens"]
     df2["ner"] = df["True"]
     df2["ner_spacy"] = df["Predicted"]
     df2["doc"] = idx
     test_df = test_df.append(df2)
      
test_df=test_df[test_df["ner"]!="-"]

#Sentence seperator
test_df.loc[(test_df["text"]==""),'ner_spacy']=""
test_df.loc[(test_df["text"]==""),'ner']=""
test_df.loc[(test_df["text"]==""),'doc']=""



with open("train_res_bilou.txt",'w+',encoding = "utf-8") as f:
  training_df.to_csv(f,sep = " ",encoding = "utf-8",index = False, header =False)
  
with open("test_res_bilou.txt",'w+',encoding = "utf-8") as f:
  test_df.to_csv(f,sep = " ",encoding = "utf-8",index = False,header = False)

Flair model

In [3]:
! pip install flair

     |████████████████████████████████| 143kB 2.9MB/s 
     |████████████████████████████████| 798kB 41.7MB/s 
     |████████████████████████████████| 655kB 44.2MB/s 
     |████████████████████████████████| 133kB 40.3MB/s 
     |████████████████████████████████| 1.0MB 36.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
Successfully built segtok mpld3 regex sqlitedict


In [4]:
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from typing import List


# columns of "gold standard" ner annotations and text
columns = {1:"ner",3:"text"}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/SAKI/Exercise_2'

# tag we want to predict
tag_type = 'ner'

downsample = 1.0 # 1.0 is full data

corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                              train_file='train_res_bilou.txt',
                                                              test_file='test_res_bilou.txt',
                                                              dev_file=None).downsample(downsample)                                                  

# Make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)
print(corpus)

2019-06-18 08:18:12,360 Reading data from /content/gdrive/My Drive/SAKI/Exercise_2
2019-06-18 08:18:12,362 Train: /content/gdrive/My Drive/SAKI/Exercise_2/train_res_bilou.txt
2019-06-18 08:18:12,367 Dev: None
2019-06-18 08:18:12,371 Test: /content/gdrive/My Drive/SAKI/Exercise_2/test_res_bilou.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


[b'<unk>', b'O', b'I-Degree', b'B-Companies_worked_at', b'I-Companies_worked_at', b'B-Degree', b'B-Designation', b'I-Designation', b'<START>', b'<STOP>']
Corpus: 31088 train + 3454 dev + 9661 test sentences


In [5]:
# Initialize embeddings
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# Initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

2019-06-18 08:18:23,224 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpow0nfo5b


100%|██████████| 160000128/160000128 [00:07<00:00, 21384086.59B/s]

2019-06-18 08:18:31,217 copying /tmp/tmpow0nfo5b to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2019-06-18 08:18:31,475 removing temp file /tmp/tmpow0nfo5b
2019-06-18 08:18:31,964 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpvcozhu3n


100%|██████████| 21494764/21494764 [00:01<00:00, 12301314.96B/s]

...

2019-06-18 08:18:40,904 copying /tmp/tmphvazziqz to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2019-06-18 08:18:41,007 removing temp file /tmp/tmphvazziqz
2019-06-18 08:18:48,522 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpyj_39wlz


100%|██████████| 73034575/73034575 [00:03<00:00, 18569269.08B/s]

2019-06-18 08:18:52,979 copying /tmp/tmpyj_39wlz to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2019-06-18 08:18:53,100 removing temp file /tmp/tmpyj_39wlz


In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/resume-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2019-06-18 09:53:05,276 ----------------------------------------------------------------------------------------------------
2019-06-18 09:53:05,278 Evaluation method: MICRO_F1_SCORE
2019-06-18 09:53:05,688 ----------------------------------------------------------------------------------------------------
2019-06-18 09:53:06,873 epoch 1 - iter 0/972 - loss 2.85868120
2019-06-18 09:53:57,488 epoch 1 - iter 97/972 - loss 1.02157452
2019-06-18 09:54:48,248 epoch 1 - iter 194/972 - loss 0.84594419
2019-06-18 09:55:40,431 epoch 1 - iter 291/972 - loss 0.76138058
2019-06-18 09:56:30,088 epoch 1 - iter 388/972 - loss 0.71832167
2019-06-18 09:57:19,477 epoch 1 - iter 485/972 - loss 0.66389715
2019-06-18 09:58:09,303 epoch 1 - iter 582/972 - loss 0.64193055
2019-06-18 09:59:00,531 epoch 1 - iter 679/972 - loss 0.62498410
2019-06-18 09:59:49,662 epoch 1 - iter 776/972 - loss 0.59946235
2019-06-18 10:00:38,680 epoch 1 - iter 873/972 - loss 0.57920926
2019-06-18 10:01:29,973 epoch 1 - iter 970/97

In [11]:
from flair.training_utils import EvaluationMetric
from typing import List, Union
from pathlib import Path
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)
model = trainer.final_test(Path("resources/taggers/resume-ner/"), True, EvaluationMetric.MICRO_F1_SCORE,32)
print(model)

2019-06-18 08:30:39,512 ----------------------------------------------------------------------------------------------------
2019-06-18 08:30:39,516 Testing using best model ...
2019-06-18 08:30:39,520 loading file resources/taggers/resume-ner/best-model.pt
2019-06-18 08:33:18,502 0.7876	0.7254	0.7552
2019-06-18 08:33:18,507 
MICRO_AVG: acc 0.6067 - f1-score 0.7552
MACRO_AVG: acc 0.6199 - f1-score 0.7648666666666667
Companies_worked_at tp: 358 - fp: 98 - fn: 158 - tn: 358 - precision: 0.7851 - recall: 0.6938 - accuracy: 0.5831 - f1-score: 0.7366
Degree     tp: 115 - fp: 31 - fn: 28 - tn: 115 - precision: 0.7877 - recall: 0.8042 - accuracy: 0.6609 - f1-score: 0.7959
Designation tp: 354 - fp: 94 - fn: 127 - tn: 354 - precision: 0.7902 - recall: 0.7360 - accuracy: 0.6157 - f1-score: 0.7621
2019-06-18 08:33:18,508 ----------------------------------------------------------------------------------------------------
0.7552
